# LU Decomposition

In [6]:
import numpy as np
import typing

import linalg_modules as linalg

## Doolittle's Method

In [7]:
def lu_decomp(coeff_mat: np.ndarray, const_vect: np.ndarray) -> typing.List[np.ndarray]:
    """
    Implements LU decomposition based on Doolittle's method.
    Returns list of [L, U, b'].
    Basically the same algorithm with row_echelon_form but
    records the multiplier value in L.
    """
    row = len(coeff_mat)
    # Create identity matrix for L
    lowertri = np.eye(row)
    uppertri = np.copy(coeff_mat)
    pivot_const = np.copy(const_vect)

    for row_idx in range(row - 1):
        # Implement partial pivoting by default
        max_row = np.argmax(abs(uppertri[row_idx:, row_idx])) + row_idx
        if max_row != row_idx:
            uppertri[[row_idx, max_row], :] = uppertri[[max_row, row_idx], :]
            pivot_const[[row_idx, max_row]] = pivot_const[[max_row, row_idx]]

        for row_idx2 in range(row_idx + 1, row):
            multiplier = uppertri[row_idx2, row_idx] / uppertri[row_idx, row_idx]
            uppertri[row_idx2, :] = (
                uppertri[row_idx2, :] - multiplier * uppertri[row_idx, :]
            )
            lowertri[row_idx2, row_idx] = multiplier

    return [lowertri, uppertri, pivot_const]

In [8]:
# Example of LU Decomposition
mat = np.array([[4.0, 3, -5], [-2, -4, 5], [8, 8, 0]])
const_vect = np.array([[-7], [11], [-8]])
lowertri, uppertri, pivot_const = lu_decomp(mat, const_vect)
print(f"L >> {lowertri}")
print(f"U >> {uppertri}")
print(f"b' >> {pivot_const}")

L >> [[ 1.    0.    0.  ]
 [-0.25  1.    0.  ]
 [ 0.5   0.5   1.  ]]
U >> [[ 8.   8.   0. ]
 [ 0.  -2.   5. ]
 [ 0.   0.  -7.5]]
b' >> [[-8]
 [11]
 [-7]]


## Find root using LU Decomposition

In [9]:
def solve_lud(coeff_mat: np.ndarray, const_vect: np.ndarray) -> np.ndarray:
    """
    Returns the root of a system of linear equations using LU decomposition.
    """
    # Use len() instead of using np.shape() and not using the column variable
    row = len(coeff_mat)
    lowertri, uppertri, pivot_const = lu_decomp(coeff_mat, const_vect)
    # Forward substitution to get vector y where,
    # Ly = b
    vect_y = np.zeros((row, 1))
    for row_idx in range(row):
        vect_y[row_idx] = (
            pivot_const[row_idx]
            - np.dot(lowertri[row_idx, :row_idx], vect_y[:row_idx, 0])
        ) / lowertri[row_idx, row_idx]
    # print(uppertri)
    # print(vect_y)
    # Backward substitution to get root vector where,
    # Ux = y
    root_vect = np.zeros((row, 1))
    for row_idx in range(row - 1, -1, -1):
        root_vect[row_idx] = (
            vect_y[row_idx, 0]
            - np.dot(uppertri[row_idx, row_idx + 1 :], root_vect[row_idx + 1 :])
        ) / uppertri[row_idx, row_idx]

    return root_vect

In [10]:
# Example of LU Decomposition
coeff_mat = np.array([[4.0, 3, -5], [-2, -4, 5], [8, 8, 0]])
const_vect = np.array([[-7], [11], [-8]])
root = solve_lud(coeff_mat, const_vect)
print(f"solution >> {root}")

solution >> [[ 1.]
 [-2.]
 [ 1.]]


# Thomas Algorithm

In [11]:
def tridiagonal(
    size: int, main_diag: float, sub_diag: float, sup_diag: float
) -> np.ndarray:
    """
    Returns a tridiagonal array with (size)x(size).
    Diagonal values are taken as parameters.
    """
    tridiag = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            if i == j:
                tridiag[i][j] = main_diag
            if j == i + 1:
                if i + 1 >= size:
                    pass
                else:
                    tridiag[i][j] = sup_diag
            if j == i - 1:
                if i - 1 < 0:
                    pass
                else:
                    tridiag[i][j] = sub_diag

    return tridiag

In [12]:
# Alternative version using numpy functions instead
def tridiagonal2(
    size: int, main_diag: float, sub_diag: float, sup_diag: float
) -> np.ndarray:
    main_diag_values = np.full(size, main_diag)
    sub_diag_values = np.full(size - 1, sub_diag)
    sup_diag_values = np.full(size - 1, sup_diag)
    tridiag = (
        np.diag(main_diag_values)
        + np.diag(sub_diag_values, k=-1)
        + np.diag(sup_diag_values, k=1)
    )
    return tridiag

In [13]:
# Example of tridiagonal matrix
tridiag = tridiagonal(10, 1, 2, 3)
print(tridiag)

[[1. 3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 1. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 1. 3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 1. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 1. 3. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 1. 3. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 1. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 1. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 1. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 1.]]


In [14]:
def thomas(tridiag: np.ndarray, const_vect: np.ndarray) -> np.ndarray:
    """
    Returns the root of a system of linear equations using the
    Thomas algorithm.
    """
    row = len(tridiag)
    for i in range(row):
        # Normalize by diagonal compound
        tridiag[i, :] = tridiag[i, :] / tridiag[i, i]
        const_vect[i] = const_vect[i] / tridiag[i, i]
        # Eliminate non-zero entries below the diagonal compound
        for i2 in range(i + 1, row):
            if tridiag[i2, i] != 0:
                tridiag[i2, :] -= tridiag[i, :] * tridiag[i2, i]
                # Subtract corresponding constant vector entries
                const_vect[i2] -= const_vect[i] * tridiag[i2, i]

    # Use backward substitution to get the root vector
    root_vect = np.zeros((row, 1))
    # Last entry is simply equated
    root_vect[-1] = const_vect[-1]
    for i in range(row - 2, -1, -1):
        root_vect[i] = const_vect[i] - tridiag[i, i + 1] * root_vect[i + 1]

    return root_vect

In [15]:
# Example of Thomas Algorithm
tridiag = tridiagonal2(10, -2.0, 1.0, 1.0)  # Should put in float values!
const_vect = np.arange(1, 11)
root = thomas(tridiag, const_vect)
print(f"solution >> {root}")
# root_veri = np.linalg.solve(tridiag, const_vect)
# print(f"solution >> {root_veri}")

solution >> [[10.]
 [18.]
 [24.]
 [28.]
 [30.]
 [30.]
 [28.]
 [24.]
 [18.]
 [10.]]


# Iterative Methods

## Jacobi Method - Matrix-wise

In [20]:
def jacobi(
    coeff_mat: np.ndarray,
    const_vect: np.ndarray,
    init_guess: np.ndarray = None,
    iterate: int = 100,
    tol: float = 1e-6,
):
    """
    Returns the root of a system of linear equations solved by Jacobi method.
    """
    row = len(coeff_mat)
    if init_guess == None:
        init_guess = np.ones(row)

    # Decompose the coefficient matrix
    diag_mat = np.diag(np.diag(coeff_mat))
    # Should set the 'k' parameter to exclude main diagonal
    lowertri = np.tril(coeff_mat, -1)
    uppertri = np.triu(coeff_mat, 1)

    root_vect = init_guess
    for _ in range(iterate):
        part1 = np.linalg.inv(diag_mat)
        part2 = const_vect - np.dot((lowertri + uppertri), root_vect)
        root_vect_new = np.dot(part1, part2)
        if np.linalg.norm(np.matmul(coeff_mat, root_vect_new) - const_vect, 2) < tol:
            return root_vect_new
        root_vect = root_vect_new

    return root_vect

In [21]:
# Example of Jacobi Method
tridiag = tridiagonal2(10, -2.0, 1.0, 1.0)  # Should put in float values!
const_vect = np.arange(1, 11)
root = jacobi(tridiag, const_vect)
print(f"solution >> {root}")

root_verify = np.linalg.solve(tridiag, const_vect)
print(f"solution verification >> {root_verify}")

solution >> [-19.60741788 -38.24600559 -54.94689553 -68.73139881 -78.62072613
 -83.61956373 -82.73246636 -74.94600749 -59.24663978 -34.60708672]
solution verification >> [-20. -39. -56. -70. -80. -85. -84. -76. -60. -35.]


## Gauss-Seidal Method

In [22]:
def gauss_seidal(
    coeff_mat: np.ndarray,
    const_vect: np.ndarray,
    init_guess: np.ndarray = None,
    iterate: int = 100,
    tol: float = 1e-6,
):
    """
    Returns the root of a system of linear equations solved by Gauss-Seidal method.
    """
    row = len(coeff_mat)
    if init_guess == None:
        init_guess = np.ones(row)

    root_vect = init_guess

    for _ in range(iterate):
        for i in range(row):
            root_vect[i] = (
                const_vect[i]
                - np.dot(coeff_mat[i, :], root_vect)
                + coeff_mat[i, i] * root_vect[i]
            ) / coeff_mat[i, i]
        if np.linalg.norm(np.matmul(coeff_mat, root_vect) - const_vect, 2) < tol:
            return root_vect

    return root_vect

In [23]:
# Example of Gauss-Seidal Method
tridiag = tridiagonal2(10, -2.0, 1.0, 1.0)  # Should put in float values!
const_vect = np.arange(1, 11)
root = gauss_seidal(tridiag, const_vect)
print(f"solution >> {root}")

solution >> [-19.99191557 -38.98511452 -55.98003479 -69.97694296 -79.97592666
 -84.9769018  -83.97963292 -75.98376386 -59.98885563 -34.99442781]
